In [1]:
from nustar_pysolar import planning, io
import astropy.units as u

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_150.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [4]:
tstart = '2017-06-10T12:00:00'
tend = '2017-06-10T20:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [5]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 348.3764402915343 deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [6]:
offset = [-190., -47.]*u.arcsec

# Loop over each orbit and see what the difference between the two methods is

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

*Looks like a fixed shift...probably some time-ephemeris issue.*

In [7]:
from astropy.coordinates import SkyCoord
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    skyfield_pos = planning.get_skyfield_position(midTime, offset, load_path='../data')
    print('SkyField Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), skyfield_pos[0], skyfield_pos[1]))
    skyfield_ephem = SkyCoord(skyfield_pos[0], skyfield_pos[1])
    sunpy_ephem = SkyCoord(sky_pos[0], sky_pos[1])
    print("")
    print("Offset between SkyField and Astropy: {} arcsec".format(skyfield_ephem.separation(sunpy_ephem).arcsec))
    print("")
         

Orbit: 0
Orbit start: 2017-06-10T11:25:40 Orbit end: 2017-06-10T12:27:30
Aim time: 2017-06-10T11:56:35 RA (deg): 78.69718876009287 deg Dec (deg): 23.0238274997264 deg
SkyField Aim time: 2017-06-10T11:56:35 RA (deg): 78.70325511331194 deg Dec (deg): 23.024266303242694 deg

Offset between SkyField and Astropy: 20.16118641902529 arcsec

Orbit: 1
Orbit start: 2017-06-10T13:02:20 Orbit end: 2017-06-10T14:04:10
Aim time: 2017-06-10T13:33:15 RA (deg): 78.76668616118683 deg Dec (deg): 23.02881188444142 deg
SkyField Aim time: 2017-06-10T13:33:15 RA (deg): 78.77275293085837 deg Dec (deg): 23.02924803267842 deg

Offset between SkyField and Astropy: 20.161074567152998 arcsec

Orbit: 2
Orbit start: 2017-06-10T14:39:00 Orbit end: 2017-06-10T15:40:50
Aim time: 2017-06-10T15:09:55 RA (deg): 78.83618771028821 deg Dec (deg): 23.033765804969477 deg
SkyField Aim time: 2017-06-10T15:09:55 RA (deg): 78.84225489427135 deg Dec (deg): 23.034199297206257 deg

Offset between SkyField and Astropy: 20.160964271832

# Okay, now check to see what the parallax does in each orbit.

### Compare Astropy/Sunpy to what you get when you correct for the orbital parallax. Every step below is 100 seconds.



In [8]:
from astropy.coordinates import SkyCoord
from datetime import timedelta
for ind, orbit in enumerate(orbits):
    midTime = orbit[0]
    while(midTime < orbit[1]):
        
        sky_pos = planning.get_sky_position(midTime, offset)

        skyfield_pos = planning.get_skyfield_position(midTime, offset, load_path='../data', parallax_corection=True)

        skyfield_ephem = SkyCoord(skyfield_pos[0], skyfield_pos[1])
        sunpy_ephem = SkyCoord(sky_pos[0], sky_pos[1])
        print('Offset between parallax-corrected positions and Astropy/Sunpy is {} arcsec'.format(
            skyfield_ephem.separation(sunpy_ephem).arcsec)
             )
        midTime += timedelta(seconds=100)
        
    break

Offset between parallax-corrected positions and Astropy/Sunpy is 28.701866244373303 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.0917263437428 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.37961960726448 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.562423247836275 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.63821137474459 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.606262228338366 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.467077830353634 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 29.222363663129972 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 28.87501922601281 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 28.429090278155133 arcsec
Offset between parallax-corrected positions and Astropy/Sunpy is 27.889731190910098 arcsec
Offs